# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import patsy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [2]:
df = pd.read_csv('previsao_de_renda.csv', index_col=[0])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               15000 non-null  object 
 1   id_cliente             15000 non-null  int64  
 2   sexo                   15000 non-null  object 
 3   posse_de_veiculo       15000 non-null  bool   
 4   posse_de_imovel        15000 non-null  bool   
 5   qtd_filhos             15000 non-null  int64  
 6   tipo_renda             15000 non-null  object 
 7   educacao               15000 non-null  object 
 8   estado_civil           15000 non-null  object 
 9   tipo_residencia        15000 non-null  object 
 10  idade                  15000 non-null  int64  
 11  tempo_emprego          12427 non-null  float64
 12  qt_pessoas_residencia  15000 non-null  float64
 13  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(3), object(6)
memory usa

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.
    

In [4]:
df['posse_de_imovel'].value_counts()

True     10143
False     4857
Name: posse_de_imovel, dtype: int64

In [5]:
patsy.dmatrix('C(posse_de_imovel, Treatment(1))', data=df)

DesignMatrix with shape (15000, 2)
  Intercept  C(posse_de_imovel, Treatment(1))[T.False]
          1                                          0
          1                                          0
          1                                          0
          1                                          0
          1                                          1
          1                                          0
          1                                          0
          1                                          0
          1                                          0
          1                                          0
          1                                          1
          1                                          0
          1                                          0
          1                                          1
          1                                          0
          1                                          0
          1                   

In [10]:
y, x = patsy.dmatrices('np.log(renda) ~ C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel, Treatment(1)) + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil) + C(tipo_residencia, Treatment(1)) + qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia', data=df)
x

DesignMatrix with shape (12427, 25)
  Columns:
    ['Intercept',
     'C(sexo)[T.M]',
     'C(posse_de_veiculo)[T.True]',
     'C(posse_de_imovel, Treatment(1))[T.False]',
     'C(tipo_renda)[T.Bolsista]',
     'C(tipo_renda)[T.Empresário]',
     'C(tipo_renda)[T.Pensionista]',
     'C(tipo_renda)[T.Servidor público]',
     'C(educacao, Treatment(2))[T.Primário]',
     'C(educacao, Treatment(2))[T.Pós graduação]',
     'C(educacao, Treatment(2))[T.Superior completo]',
     'C(educacao, Treatment(2))[T.Superior incompleto]',
     'C(estado_civil)[T.Separado]',
     'C(estado_civil)[T.Solteiro]',
     'C(estado_civil)[T.União]',
     'C(estado_civil)[T.Viúvo]',
     'C(tipo_residencia, Treatment(1))[T.Aluguel]',
     'C(tipo_residencia, Treatment(1))[T.Com os pais]',
     'C(tipo_residencia, Treatment(1))[T.Comunitário]',
     'C(tipo_residencia, Treatment(1))[T.Estúdio]',
     'C(tipo_residencia, Treatment(1))[T.Governamental]',
     'qtd_filhos',
     'idade',
     'tempo_emprego',
   

In [11]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Tue, 25 Jul 2023   Prob (F-statistic):               0.00
Time:                        01:25:53   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.6093      0.219     30.214      0.000       6.181       7.038
C(sexo)[T.M]                                          0.7874      0.015     53.723      0.000       0.759       0.816
C(posse_de_veiculo)[T.True]                           0.0441      0.014      3.119      0.002       0.016       0.072
C(posse_de_imovel, Treatment(1))[T.False]            -0.0829      0.014     -5.926      0.000      -0.110      -0.055
C(tipo_renda)[T.Bolsista]                             0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda)[T.Empresário]                           0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda)[T.Pensionista]                         -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda)[T.Servidor público]                     0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment(2))[T.Primário]                 0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment(2))[T.Pós graduação]            0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment(2))[T.Superior completo]        0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment(2))[T.Superior incompleto]     -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil)[T.Separado]                           0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil)[T.Solteiro]                           0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil)[T.União]                             -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil)[T.Viúvo]                              0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment(1))[T.Aluguel]           0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment(1))[T.Com os pais]       0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment(1))[T.Governamental]    -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos                                           -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                                 0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego  

In [8]:
y, x = patsy.dmatrices('np.log(renda) ~ C(posse_de_veiculo) + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil) + C(tipo_residencia, Treatment(1))', data=df)
x

DesignMatrix with shape (15000, 19)
  Columns:
    ['Intercept',
     'C(posse_de_veiculo)[T.True]',
     'C(tipo_renda)[T.Bolsista]',
     'C(tipo_renda)[T.Empresário]',
     'C(tipo_renda)[T.Pensionista]',
     'C(tipo_renda)[T.Servidor público]',
     'C(educacao, Treatment(2))[T.Primário]',
     'C(educacao, Treatment(2))[T.Pós graduação]',
     'C(educacao, Treatment(2))[T.Superior completo]',
     'C(educacao, Treatment(2))[T.Superior incompleto]',
     'C(estado_civil)[T.Separado]',
     'C(estado_civil)[T.Solteiro]',
     'C(estado_civil)[T.União]',
     'C(estado_civil)[T.Viúvo]',
     'C(tipo_residencia, Treatment(1))[T.Aluguel]',
     'C(tipo_residencia, Treatment(1))[T.Com os pais]',
     'C(tipo_residencia, Treatment(1))[T.Comunitário]',
     'C(tipo_residencia, Treatment(1))[T.Estúdio]',
     'C(tipo_residencia, Treatment(1))[T.Governamental]']
  Terms:
    'Intercept' (column 0)
    'C(posse_de_veiculo)' (column 1)
    'C(tipo_renda)' (columns 2:6)
    'C(educacao, Treat

In [9]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     41.29
Date:                Tue, 25 Jul 2023   Prob (F-statistic):          1.98e-142
Time:                        01:23:52   Log-Likelihood:                -19061.
No. Observations:               15000   AIC:                         3.816e+04
Df Residuals:                   14981   BIC:                         3.831e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             8.1472      0.014    581.540      0.000       8.120       8.175
C(posse_de_veiculo)[T.True]                           0.2273      0.015     15.287      0.000       0.198       0.256
C(tipo_renda)[T.Bolsista]                             0.2378      0.288      0.826      0.409      -0.327       0.803
C(tipo_renda)[T.Empresário]                           0.0367      0.018      2.072      0.038       0.002       0.071
C(tipo_renda)[T.Pensionista]                         -0.2609      0.020    -12.765      0.000      -0.301      -0.221
C(tipo_renda)[T.Servidor público]                     0.1619      0.026      6.148      0.000       0.110       0.213
C(educacao, Treatment(2))[T.Primário]                 0.0571      0.068      0.840      0.401      -0.076       0.190
C(educacao, Treatment(2))[T.Pós graduação]           -0.1961      0.170     -1.156      0.248      -0.529       0.136
C(educacao, Treatment(2))[T.Superior completo]        0.0688      0.015      4.515      0.000       0.039       0.099
C(educacao, Treatment(2))[T.Superior incompleto]     -0.1062      0.037     -2.847      0.004      -0.179      -0.033
C(estado_civil)[T.Separado]                          -0.0284      0.030     -0.932      0.351      -0.088       0.031
C(estado_civil)[T.Solteiro]                          -0.0686      0.022     -3.085      0.002      -0.112      -0.025
C(estado_civil)[T.União]                             -0.0968      0.028     -3.492      0.000      -0.151      -0.042
C(estado_civil)[T.Viúvo]                             -0.1050      0.034     -3.050      0.002      -0.172      -0.038
C(tipo_residencia, Treatment(1))[T.Aluguel]           0.0367      0.063      0.587      0.557      -0.086       0.159
C(tipo_residencia, Treatment(1))[T.Com os pais]      -0.1003      0.034     -2.923      0.003      -0.168      -0.033
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.2892      0.109     -2.664      0.008      -0.502      -0.076
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.3429      0.095      3.603      0.000       0.156       0.529
C(tipo_residencia, Treatment(1))[T.Governamental]    -0.0224      0.041     -0.542      0.588      -0.103       0.059
==============================================================================
Omnibus:                      253.765   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              290.673
Skew:                           0.274   Prob(JB):                     7.61e-64
Kurtosis:                       3.406   Cond. No.                         49.5
==============================================================================

Notes:
[1] S

# Aparentemente, o modelo me pareceu pior dado à diminuição do R²